In [20]:
# False or misleading info that is reported as news is called fake news.This is to harm someoneor something's reputation or to profit through advertising.  

import pandas as pd#data manipulation and analysis library in Python.#
import numpy as np #handling arrays and mathematical(numerical) operations.#
import re
import string
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import classification_report


In [21]:
data_fake = pd.read_csv('Fake.csv')
data_true = pd.read_csv('True.csv')


In [22]:
data_fake['class'] = 1  # Fake news  ,new class column has been added to data set
data_true['class'] = 0  # Not fake news

In [23]:
data_fake.head() #first 5 rows

,title,text,subject,date,class
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",1
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",1
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",1
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",1
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",1


In [24]:
data_true.head()

,title,text,subject,date,class
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",0
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",0
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",0
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",0
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",0


In [25]:
data_fake.shape,data_true.shape

((23481, 5), (21417, 5))

In [26]:
data = pd.concat([data_fake, data_true], ignore_index=True)
data = data.sample(frac=1, random_state=42).reset_index(drop=True)  # Shuffle the data


In [27]:
data.isnull().sum() 

title      0
text       0
subject    0
date       0
class      0
dtype: int64

In [28]:
data.head()


,title,text,subject,date,class
0,Ben Stein Calls Out 9th Circuit Court: Committ...,"21st Century Wire says Ben Stein, reputable pr...",US_News,"February 13, 2017",1
1,Trump drops Steve Bannon from National Securit...,WASHINGTON (Reuters) - U.S. President Donald T...,politicsNews,"April 5, 2017",0
2,Puerto Rico expects U.S. to lift Jones Act shi...,(Reuters) - Puerto Rico Governor Ricardo Rosse...,politicsNews,"September 27, 2017",0
3,OOPS: Trump Just Accidentally Confirmed He Le...,"On Monday, Donald Trump once again embarrassed...",News,"May 22, 2017",1
4,Donald Trump heads for Scotland to reopen a go...,"GLASGOW, Scotland (Reuters) - Most U.S. presid...",politicsNews,"June 24, 2016",0


In [29]:
data.columns

Index(['title', 'text', 'subject', 'date', 'class'], dtype='object')

In [30]:
data = data.drop(['title','subject','date'],axis=1)

In [31]:
data.columns

Index(['text', 'class'], dtype='object')

In [32]:
def clean_text(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('\\W', ' ', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', ' ', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

In [33]:
data['text'] = data['text'].apply(clean_text)

In [34]:
X_train, X_test, y_train, y_test = train_test_split(data['text'], data['class'], test_size=0.2, random_state=42)

In [35]:
vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)


In [36]:
models = {
    "Logistic Regression": LogisticRegression(),
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(n_estimators=100),
    "Gradient Boosting": GradientBoostingClassifier(n_estimators=100)
}

In [37]:
for name, model in models.items():
    print(f"Training {name}...")
    model.fit(X_train_vectorized, y_train)

    # Prediction values of each algorithm
    y_pred_train = model.predict(X_train_vectorized)
    y_pred_test = model.predict(X_test_vectorized)
    train_accuracy = np.mean(y_pred_train == y_train)
    test_accuracy = np.mean(y_pred_test == y_test)
    print(f"{name} Train Accuracy: {train_accuracy:.2f}, Test Accuracy: {test_accuracy:.2f}")

Training Logistic Regression...
Logistic Regression Train Accuracy: 0.99, Test Accuracy: 0.98
Training Decision Tree...
Decision Tree Train Accuracy: 1.00, Test Accuracy: 0.99
Training Random Forest...
Random Forest Train Accuracy: 1.00, Test Accuracy: 1.00
Training Gradient Boosting...
Gradient Boosting Train Accuracy: 1.00, Test Accuracy: 0.99


In [40]:
# Manual testing function
def manual_testing(news):
    cleaned_news = clean_text(news)
    news_vectorized = vectorizer.transform([cleaned_news])
    
    for name, model in models.items():
        prediction = model.predict(news_vectorized)[0]
        prob_fake = model.predict_proba(news_vectorized)[0][1]  # Probability of being fake news
        print(f"{name} Prediction: {'Fake News' if prediction == 1 else 'Not Fake News'} with probability {prob_fake:.2f}")

        


In [42]:
news_input = input("Enter a news article: ")
manual_testing(news_input)

Enter a news article: EU's Juncker says ready to act over new U.S. sanctions on Russia,"BRUSSELS (Reuters) - European Commission President Jean-Claude Juncker said on Wednesday the European Union was ready to act “within a matter of days” if proposed new U.S. sanctions on Russia undermined the bloc’s energy security. The U.S. House of Representatives voted overwhelmingly on Tuesday to slap new sanctions on Russia, which could affect European firms. “The U.S. bill could have unintended unilateral effects that impact the EU’s energy security interests,” Juncker said in a statement after a meeting of European Commissioners. “This is why the Commission concluded today that if our concerns are not taken into account sufficiently, we stand ready to act appropriately within a matter of days. America first cannot mean that Europe’s interests come last.” ",politicsNews,"July 26, 2017 "
Logistic Regression Prediction: Not Fake News with probability 0.05
Decision Tree Prediction: Not Fake News wi